# DTSA 5509 Final Project
## Predicting Walkability of U.S. Counties

#### Rahul Cheeniyil
#### 12-11-2024

In this project, I use the Walkability Index dataset ([published by the U.S. Environmental Protection Agency](https://catalog.data.gov/dataset/walkability-index7)) to build and test supervised learning models to predict walkability of U.S. Census Blocks. Predicting walkability is important for allocating infrastructure funding for areas with low pedestrian accessibility. Improving city walkability can promote for public health and safety as well as reduce automobile traffic and improve environmental health due to reduced emissions.

The models I will use are multilinear regression and gradient boosting. In this report, I will walk through the data I use, how I intend to use it, data cleaning, and exploratory data analysis prior to building the models themselves. Model performance will be assessed at the end of the report.

### Data
The data consists of various metrics from Censis 2019 block groups. The data is accessed by a publicly available API. The code cell below loads the data via REST API requests and loads it into a Pandas DataFrame. The resulting DataFrame has 220,134 rows and 182 columns. Each row corresponds to a U.S. Census Block and each column responds to a metric associated with that block. The column "NatWalkInd" is the National Walkability Index score assigned to the block. This is the metric I will be predicting with machine learning models. All other columns reflect various metrics/statistics associated with the block. These 181 columns can be grouped into several categories.
- Geographic Identifiers: Spatial and administrative context for identifying the blocks
- Population and Demographics: Population of the block and various demographic distributions.
- Automobile Ownership Statistics: Car ownership/dependency of residents including percentage of households owning different numbers of vehicles.
- Employment and Wages: Job distributions and wage levels.
- Land Use and Density: Metrics related to the land itself.
- Transportation: Proximity to destinations and transit.
- Environmental Metrics: Assessments of emissions and vehicle usage.
- Geographic Measurements: Geometric characterization of the land.

In [25]:
import requests
import pandas as pd

# Publicly accessibly Esri REST API Endpoint
api_url = "https://geodata.epa.gov/arcgis/rest/services/OA/WalkabilityIndex/MapServer/0/query"

params = {
    "where": "1=1",  # Select all records
    "outFields": "*",  # Retrieve all fields
    "f": "json",  # Response format
    "resultRecordCount": 10000,  # Limit to 10000 records per request to avoid rate limits
    "returnGeometry": True,  # Include geographic data such as area/perimeter of census blocks
}

# Fetch the data in chunks
def fetch_data(api_url, params):
    offset = 0
    all_data = []

    while True:
        params["resultOffset"] = offset
        response = requests.get(api_url, params=params)

        if response.status_code == 200:
            data = response.json()
            features = data.get("features", [])
            
            if not features:
                break  # No more records
            
            all_data.extend([feature["attributes"] for feature in features])
            
            offset += len(features)
        else:
            print(f"Error: {response.status_code}")
            break

    return pd.DataFrame(all_data)

walkability_data = fetch_data(api_url, params)  # This took me about 7 minutes to run.

print(walkability_data.info)

# Report memory usage in MB
memory_B = walkability_data.memory_usage(deep=True).sum()
memory_MB = round(memory_B/1024/1024, 3)
print(f"The Complete DataFrame size is {memory_MB:,} MB.")


<bound method DataFrame.info of              GEOID10       GEOID20 STATEFP COUNTYFP TRACTCE BLKGRPCE   CSA   
0       481130078254  481130078254      48      113  007825        4   206  \
1       481130078252  481130078252      48      113  007825        2   206   
2       481130078253  481130078253      48      113  007825        3   206   
3       481130078241  481130078241      48      113  007824        1   206   
4       481130078242  481130078242      48      113  007824        2   206   
...              ...           ...     ...      ...     ...      ...   ...   
220129  780309610002  780309610002      78      030  961000        2  None   
220130  780309610003  780309610003      78      030  961000        3  None   
220131  780309610005  780309610005      78      030  961000        5  None   
220132  780309607003  780309607003      78      030  960700        3  None   
220133  780309602002  780309602002      78      030  960200        2  None   

                        CSA_Nam

### Data Cleaning
Upon examining the column data, it's fairly clear that not all are needed to build a model. Primary features of interest are going to be quantifiable, meaning we can drop all of our non-numeric datatype columns. Additionally, examining our column groupings, we likely don't need any administrative information or other identifiers, so it's safe to drop those columns. I also drop redudant columns that effecively show duplicate data that is presented differently. This could bias the models to more strongly depends on duplicated data. We can also discard any columns providing a "score" other than our target dependent variable "NatWalkInd." Scores are results from assessments and are not realistic to include in a model that is meant to infer walkability based on measurable metrics. From the remaining dataframe, I drop any rows with None or NaN values to ensure useability of data. After these steps, the data should be clean and usable for building a model. Many columns still remain, but those will be filtered down further after investigating which features best suit my models through EDA.

Remaining data size is 217,181 rows and 146 columns taking up 243.574 MB in memory.

In [26]:
# Remove columns with non-numeric values
numeric_walkability_data = walkability_data.select_dtypes(include=['number'])

# Manually drop columns with data irrelevant to model training
columns_to_drop = [
    "OBJECTID",  # Unique object identifier
    "Shape_Length",  # Metadata about polygons for mapping
    "Shape_Area",
    "Households",  # Redundant general counts already covered by more detailed columns
    "Residents",
    "Workers_1",
    "Drivers",
    "Vehicles",
    "SLC_score",  # Scores that are higher-level aggregations - not independent variables
    "D2A_Ranked",
    "D2B_Ranked",
    "D3B_Ranked",
    "D4A_Ranked",
    "B_C_*",  # Coefficients derived from statistical or predictive models of the population
    "B_N_*",
    "C_R_*",
    "GHG_per_worker", # Metrics focused on vehicle usage/emissions, likely resulting from vehicle ownership 
    "Annual_GHG",
    "NonCom_VMT_Per_Worker",
    "Com_VMT_Per_Worker",
    "VMT_per_worker",
    "logd1a",  # Log/Scaled transformations of existing variables
    "logd1c",
    "logd3aao",
    "logd3apo",
    "logd4d",
    "d4bo25",
    "d5dei_1"
]

filtered_walkability_data = numeric_walkability_data.drop(columns=columns_to_drop, errors='ignore')

# Filter out rows with missing data
cleaned_walkability_data = filtered_walkability_data.dropna()

print(cleaned_walkability_data.info)

memory_B = cleaned_walkability_data.memory_usage(deep=True).sum()
memory_MB = round(memory_B/1024/1024, 3)
print(f"The Cleaned DataFrame size is {memory_MB:,} MB.")

del walkability_data, numeric_walkability_data, filtered_walkability_data  # Free up memory taken by large intermediate DataFrames

<bound method DataFrame.info of         CBSA_POP  CBSA_EMP  CBSA_WRK       Ac_Total    Ac_Water   
0        7189384   3545715   3364458      73.595028    0.000000  \
1        7189384   3545715   3364458     119.829909    0.000000   
2        7189384   3545715   3364458      26.367053    0.000000   
3        7189384   3545715   3364458     119.060687    0.000000   
4        7189384   3545715   3364458     169.927211    0.000000   
...          ...       ...       ...            ...         ...   
217177     80610     39570     37773     193.420737    0.000000   
217178     80610     39570     37773     495.303549   44.102902   
217179     80610     39570     37773     126.513965    0.000000   
217180         0         0         0  367912.651157  810.976517   
217181         0         0         0  914821.703629  423.499748   

              Ac_Land        Ac_Unpr  TotPop  CountHU     HH  ...   
0           73.595028      73.595028    1202    460.0  423.0  ...  \
1          119.829909    

### Exploratory Data Analysis (EDA)

